In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
from scipy.io import savemat
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},   

In [4]:
# All functions used below
def derivtransfer(FPs,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanhp':
        derive=1/np.cosh(xe)**2
        derivi=1/np.cosh(xi)**2
        return np.array([derive,derivi])
def tanhpfunc(x):
    return np.tanh(x)+1.0
def checkXbase(FPs,J,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanhp':
        derxe=-xe+tanhpfunc(xe)*J[0,0]+tanhpfunc(xi)*J[0,1]
        derxi=-xi+tanhpfunc(xe)*J[1,0]+tanhpfunc(xi)*J[1,1]
        return (derxe,derxi)
    elif type=='tanh':
        derxe=-xe+np.tanh(xe)*J[0,0]+np.tanh(xi)*J[0,1]
        derxi=-xi+np.tanh(xe)*J[1,0]+np.tanh(xi)*J[1,1]
        return (derxe,derxi)
def FPfunc(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=np.tanh(x0+x1)+1
        resp1=np.tanh(x0-x1)+1
        return [
            x0-JE*resp0+JI*resp1,
            x1-a*resp0-b*resp1
        ]  
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=(np.tanh(x0+x1))
        resp1=(np.tanh(x0-x1))
        return [
            x0-JE*resp0+JI*resp1,
            x1-a*resp0-b*resp1
        ]        


def FPfuncSVD(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        n[:,0]*=sv[0]
        n[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=np.tanh(x0*m[0,0]+x1*m[0,1])+1
        resp1=np.tanh(x0*m[1,0]+x1*m[1,1])+1
        epszero=1e-6
        if sv[1]<-epszero:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1
            ]
        else:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1-n[0,1]*resp0-n[1,1]*resp1
            ]
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        n[:,0]*=sv[0]
        n[:,1]*=sv[1]
        # print('sigma2:',sv[1])
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=(np.tanh(x0*m[0,0]+x1*m[0,1])+0.0)
        resp1=(np.tanh(x0*m[1,0]+x1*m[1,1])+0.0)
        epszero=1e-6
        if sv[1]<-epszero:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1
            ]
        else:
            return [
                x0-n[0,0]*resp0-n[1,0]*resp1,
                x1-n[0,1]*resp0-n[1,1]*resp1
            ]

    
def FPXfunc(x,*data):
    JE,JI,a,b,tfunc=data
    # calculate connectivity matrix
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        respxE=tanhpfunc(x0)*Jt[0,0]+tanhpfunc(x1)*Jt[0,1]
        respxI=tanhpfunc(x0)*Jt[1,0]+tanhpfunc(x1)*Jt[1,1]
        return [
            x0-respxE,
            x1-respxI
        ]  
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        respxE=np.tanh(x0)*Jt[0,0]+np.tanh(x1)*Jt[0,1]
        respxI=np.tanh(x0)*Jt[1,0]+np.tanh(x1)*Jt[1,1]
        return [
            x0-respxE,
            x1-respxI
        ]  
        

In [5]:
def dynkappa(M,N,kappaFP,kappaP,timeparams,type='tanhp',express='EI'):
    tfinal,dt=timeparams[0],timeparams[1]
    nt = int(tfinal/dt)
    kappadyn=np.zeros((2,nt))
    kappadyn[:,0]=kappaFP+kappaP
    xactdyn = np.zeros_like(kappadyn)
    xactdyn[:,0]=kappadyn[0,0]*M[:,0]+kappadyn[1,0]*M[:,1]
    eps=1e-5
    if type=='tanhp':
        # check the FP
        xFP=kappaFP[0]*M[:,0]+kappaFP[1]*M[:,1]
        phisyn=np.zeros_like(kappaFP)
        for i in range(2):
            phisyn[i]=tanhpfunc(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        devkappa=-np.reshape(kappaFP,(2,1))+np.reshape(N.T@phisyn,(2,1))
        if np.sum(devkappa**2)>eps:
            print(express,' error of kappa FPs:',np.sum(devkappa**2))
            print('FP error')
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(tanhpfunc(xact),(2,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]
    elif type=='tanh':
        # check the FP
        xFP=kappaFP[0]*M[:,0]+kappaFP[1]*M[:,1]
        phisyn=np.zeros_like(kappaFP)
        for i in range(2):
            phisyn[i]=np.tanh(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        devkappa=-np.reshape(kappaFP,(2,1))+np.reshape(N.T@phisyn,(2,1))
        if np.sum(devkappa**2)>eps:
            print(express,' error of kappa FPs:',np.sum(devkappa**2))
            print('FP error')
        for it in range(1,nt):
            xact = kappadyn[0,it-1]*M[:,0]+kappadyn[1,it-1]*M[:,1]
            phisyn=np.reshape(np.tanh(xact),(2,1))
            deltakappa=dt*(-kappadyn[:,it-1]+np.squeeze(N.T@phisyn))
            kappadyn[:,it]=kappadyn[:,it-1]+np.squeeze(deltakappa)
            xactdyn[:,it]=kappadyn[0,it]*M[:,0]+kappadyn[1,it]*M[:,1]

    return (kappadyn,xactdyn,devkappa)

def dynxact(J,xFP,xP,timeparams,type='tanhp'):
    tfinal,dt=timeparams[0],timeparams[1]
    nt = int(tfinal/dt)
    xactdyn=np.zeros((2,nt))
    # print(xFP)
    xactdyn[:,0]=xFP+xP
    eps=1e-5
    if type=='tanhp':
        # check the FP
        phisyn=np.zeros_like(xFP)
        for i in range(2):
            phisyn[i]=tanhpfunc(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        # print(phisyn)
        devxact=-np.reshape(xFP,(2,1))+np.reshape(J@phisyn,(2,1))
        if np.sum(devxact**2)>eps:
            print('error of FPs:',np.sum(devxact**2))
            print('x activity FP error')
        for it in range(1,nt):
            phisyn=np.reshape(tanhpfunc(xactdyn[:,it-1]),(2,1))
            deltaxact=dt*(-xactdyn[:,it-1]+np.squeeze(J@phisyn))
            xactdyn[:,it]=xactdyn[:,it-1]+deltaxact
    elif type=='tanh':
        # check the FP
        phisyn=np.zeros_like(xFP)
        for i in range(2):
            phisyn[i]=np.tanh(xFP[i])
        phisyn=np.reshape(phisyn,(2,1))
        # print(phisyn)
        devxact=-np.reshape(xFP,(2,1))+np.reshape(J@phisyn,(2,1))
        if np.sum(devxact**2)>eps:
            print('error of FPs:',np.sum(devxact**2))
            print('x activity FP error')
        for it in range(1,nt):
            phisyn=np.reshape(np.tanh(xactdyn[:,it-1]),(2,1))
            deltaxact=dt*(-xactdyn[:,it-1]+np.squeeze(J@phisyn))
            xactdyn[:,it]=xactdyn[:,it-1]+deltaxact
    return (xactdyn,devxact)


In [11]:
# 2 by 2 connectivity matrix, rank-1 structure
# transfer function \phi(x)=tanh(x)
'''
J = np.zeros((2,2))
'''
JE,JI,a,b=1.0,1.2,0.0,0.0#0.40,0.30
jeseries = np.linspace(0.0,2.8,num =80)  
nlen=len(jeseries)
M=np.array([[1,1],[1,-1]])
xFPseries = np.zeros((nlen,2,2))
svdvalues = np.zeros((nlen,2))
svdvec = np.zeros((nlen,2,4))
kappaMN,kappamnSVD=np.zeros((nlen,2)),np.zeros((nlen,2))
epsErr=1e-5
for idxje,JE in enumerate(jeseries):
    # calculate SVD
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
    if (np.min(Jt[:,0])<0.0) or (np.max(Jt[:,1])>0.0):
        xFPseries[idxje,:,:]=np.nan
        kappaMN[idxje,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue   
    lvec,sv,rvech=la.svd(Jt)
    svdvalues[idxje,:]=sv
    m,n=lvec,rvech.T
            
    n[:,0]*=sv[0]
    n[:,1]*=sv[1]
    if n[0,0]<0:
        n[:,0]*=(-1)   
        m[:,0]*=(-1)
    if n[0,1]<0:
        n[:,1]*=(-1)   
        m[:,1]*=(-1)
    svdvec[idxje,:,:2]=m
    svdvec[idxje,:,2:]=n

    N=np.array([[JE,a],[-JI,b]])
    data=(JE,JI,a,b,'tanh')
    x0=[10.0,-2.0]
    results = fsolve(FPfunc,x0,data)
    kappaMN[idxje,:]=results
    xFP= M@np.reshape(results,(2,1))
    # check the solution
    errE,errI=checkXbase(xFP,Jt,type='tanh')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,0,:]=np.nan
        kappaMN[idxje,:]=np.nan
        continue
    xFPseries[idxje,0,:]=xFP[:,0]
    
    resultSVD= fsolve(FPfuncSVD,x0,data)
    kappamnSVD[idxje,:]=resultSVD
    xFPSVD= m@np.reshape(resultSVD,(2,1))
    # check the solution
    errE,errI=checkXbase(xFPSVD,Jt,type='tanh')
    if errE>epsErr or errI>epsErr:
        xFPseries[idxje,1,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue
    xFPseries[idxje,1,:]=xFPSVD[:,0]
    # check eig
    Xderive=derivtransfer(xFPSVD,type='tanhp')
    Jcheck=Jt.copy()
    Jcheck[:,0]*=Xderive[0]
    Jcheck[:,1]*=Xderive[1]
    eigv,eigvec=la.eig(Jcheck)
    if eigv[0]>1.0:
        xFPseries[idxje,:,:]=np.nan
        kappaMN[idxje,:]=np.nan
        kappamnSVD[idxje,:]=np.nan
        continue

In [12]:
# testing singular value and ratio of singular vectors
JE,JI,a,b=3.6,1.2,-0.5,-0.15
Jt = np.zeros((2,2))
Jt[:,0],Jt[:,1]=JE,-JI
Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
a1,a2 = np.sqrt((JE+b)**2+(JI+a)**2),np.sqrt((JE-b)**2+(JI-a)**2)
sigm1_theo,sigm2_theo=1/np.sqrt(2)*(a1+a2),1.0/np.sqrt(2)*(a1-a2)
lvec,sigv,rvech=la.svd(Jt)
l=a1*a2
v1_ratio=-(JE**2+a**2-JI**2-b**2)+2*(JE*JI-a*b)-l
v1_ratio/=(JE**2+a**2-JI**2-b**2)+2*(JE*JI-a*b)-l
v2_ratio=-(JE**2+a**2-JI**2-b**2)+2*(JE*JI-a*b)+l
v2_ratio/=(JE**2+a**2-JI**2-b**2)+2*(JE*JI-a*b)+l

u1_ratio=(JE**2+JI**2)-(a**2+b**2)-2*(-a*JE+b*JI)+l
u1_ratio/=(JE**2+JI**2)-(a**2+b**2)+2*(-a*JE+b*JI)+l
u2_ratio=(JE**2+JI**2)-(a**2+b**2)-2*(-a*JE+b*JI)-l
u2_ratio/=(JE**2+JI**2)-(a**2+b**2)+2*(-a*JE+b*JI)-l

v1ratio_num=rvech[0,0]/rvech[0,1]
v2ratio_num=rvech[1,0]/rvech[1,1]

u1ratio_num=lvec[0,0]/lvec[1,0]
u2ratio_num=lvec[0,1]/lvec[1,1]

print('compare:')
print('singular values, numerical:',sigv,'; theory:',sigm1_theo,sigm2_theo)
print('left vector, numerical:',u1ratio_num,u2ratio_num,'; theory:',u1_ratio,u2_ratio)
print('right vector, numerical:',v1ratio_num,v2ratio_num,'; theory:',v1_ratio,v2_ratio)

sigm_l1,sigm_l2=np.sqrt(2*(JE**2+JI**2)),np.sqrt(2/(JE**2+JI**2))*(JI*a+JE*b)
print('approx sigma:',sigm_l1,sigm_l2)



compare:
singular values, numerical: [5.40062679 0.42217322] ; theory: 5.400626794448373 -0.42217321929072155
left vector, numerical: 0.7966214668493156 -1.255301346516631 ; theory: 0.7966214668493157 -1.2553013465166323
right vector, numerical: -3.0909033890357738 0.3235300085881869 ; theory: -3.090903389035773 0.3235300085881871
approx sigma: 5.366563145999495 -0.4248529157249601


In [13]:
fig=plt.figure()

ax1=fig.add_subplot(131)
ax1.plot(jeseries,xFPseries[:,0,0],'r')
ax1.plot(jeseries,xFPseries[:,0,1],'b')
ax1.set_xlabel(r'$J_E$',fontsize=14)
ax1.set_ylabel(r'$x_{E/I}$',fontsize=14)
# ax2=fig.add_subplot(131)
# ax2.plot(jeseries,xFPseries[:,1,0],'r')
# ax2.plot(jeseries,xFPseries[:,1,1],'b')

ax3=fig.add_subplot(132)
ax3.plot(jeseries,kappaMN[:,0],'r')
ax3.plot(jeseries,kappaMN[:,1],'b')
ax3.set_xlabel(r'$J_E$',fontsize=14)
ax3.set_ylabel(r'$\kappa^{EI}$',fontsize=14)
ax3.set_title(r'$J_I=$'+str(JI)+', $a,\ b=$'+str(a)+','+str(b),fontsize=16)

ax4=fig.add_subplot(133)
ax4.plot(jeseries,kappamnSVD[:,0],'r')
ax4.plot(jeseries,kappamnSVD[:,1],'b')
ax4.set_xlabel(r'$J_E$',fontsize=14)
ax4.set_ylabel(r'$\kappa^{SVD}$',fontsize=14)

Text(0, 0.5, '$\\kappa^{SVD}$')

In [14]:
# reversely calculate kappaSVD
def RevfuncSVD(x,*data):
    m,xFP,tfunc=data
    if tfunc=='tanhp':
        k0 = float(x[0])
        k1 = float(x[1])
        rate1=tanhpfunc(xFP[0])
        rate2=tanhpfunc(xFP[1])
    
        return [
            k0*m[0,0]+k1*m[0,1]-xFP[0],
            k0*m[1,0]+k1*m[1,1]-xFP[1]
        ]

In [15]:
# Noncline analysis --- intersection of two lines
# JE,JI,a,b=2.2-0.7,0.8-0.7,0.15,0.05


## fix large JE-JI and a+b
# amb, JI  = 1.5,1.5 # fixed here
# JEmI,apb = -0.3,-0.3
# JE=JI+JEmI

JEmI,JEpI  = 1.5,2.5# fixed here
amp,apb = 1.2,-0.3
JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
a,b=(apb+amb)/2.0,(apb-amb)/2.0
print('JE',JE)
# a,b=-JE,JI
Jt = np.zeros((2,2))
Jt[:,0],Jt[:,1]=JE,-JI
Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
# # Use neuronal responses X
# xE,xI=np.linspace(-5.0,5.0,320),np.linspace(-5.0,5.0,320)
# [XE,XI]=np.meshgrid(xE,xI)
# # Surface of rhs in ODEs
# rhsE,rhsI=Jt[0,0]*tanhpfunc(XE)+Jt[0,1]*tanhpfunc(XI),Jt[1,0]*tanhpfunc(XE)+Jt[1,1]*tanhpfunc(XI)
# # rhsE,rhsI=Jt[0,0]*np.tanh(XE)+Jt[0,1]*np.tanh(XI),Jt[1,0]*np.tanh(XE)+Jt[1,1]*np.tanh(XI)
# epsdis = 1*1e-4#((xE[1]-xE[0])/5.0)**2
# # finding the intersection
# distrhsE=(rhsE-XE)**2
# intersectE=np.where(distrhsE<epsdis)
# distrhsI=(rhsI-XI)**2
# intersectI=np.where(distrhsI<epsdis)

# ## plot intersection line
# fig=plt.figure()
# ax1=plt.axes()
# ax1.scatter(xE[intersectI[1]],xI[intersectI[0]],c='blue',s=1,label=r'Nullcline $x_I$')
# ax1.scatter(xE[intersectE[1]],xI[intersectE[0]],c='red',s=1,label=r'Nullcline $x_E$')
# ax1.set_xlabel(r'$x_E$')
# ax1.set_ylabel(r'$x_I$')
# ax1.set_title(r'$J_E=$'+str(JE)+r',$J_I=$'+str(JI)+r',$a=$'+str(a)+r',$b=$'+str(b))
# plt.axis('square')
# plt.legend()
# plt.grid()

NameError: name 'amb' is not defined

In [16]:
# Use kappa EI
# M and N
JEmI,JEpI  = -2.1,2.1# fixed here
amb,apb = 1.2,-0.3
apbs=np.linspace(-0.3,0.3,24)
for idxp, apb in enumerate(apbs):
    JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
    a,b=(apb+amb)/2.0,(apb-amb)/2.0
    # print('JE',JE)
    # a,b=-JE,JI
    Jt = np.zeros((2,2))
    Jt[:,0],Jt[:,1]=JE,-JI
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
    M,N = np.array([[1,1],[1,-1]]),np.array([[JE,a],[-JI,b]])
    kappaei1,kappaei2=np.linspace(-3.0,3.0,301),np.linspace(-3.0,3.0,301)
    [kappaEI1,kappaEI2]=np.meshgrid(kappaei1,kappaei2)
    # Surface of rhs in ODEs
    XE,XI=kappaEI1*M[0,0]+kappaEI2*M[0,1],kappaEI1*M[1,0]+kappaEI2*M[1,1]
    rhs1,rhs2=N[0,0]*tanhpfunc(XE)+N[1,0]*tanhpfunc(XI),N[0,1]*tanhpfunc(XE)+N[1,1]*tanhpfunc(XI)
    # rhsE,rhsI=Jt[0,0]*np.tanh(XE)+Jt[0,1]*np.tanh(XI),Jt[1,0]*np.tanh(XE)+Jt[1,1]*np.tanh(XI)
    epsdis = 1*1e-5#((xE[1]-xE[0])/5.0)**2
    # finding the intersection
    distrhs1=(rhs1-kappaEI1)**2
    intersect1=np.where(distrhs1<epsdis)
    distrhs2=(rhs2-kappaEI2)**2
    intersect2=np.where(distrhs2<epsdis)

    ## plot intersection line
    fig=plt.figure()
    ax1=plt.axes()
    ax1.scatter(kappaei1[intersect1[1]],kappaei2[intersect1[0]],c='red',s=1,label=r'Nullcline $\kappa_1^{EI}$')
    ax1.scatter(kappaei1[intersect2[1]],kappaei2[intersect2[0]],c='blue',s=1,label=r'Nullcline $\kappa_2^{EI}$')
    ax1.set_xlabel(r'$\kappa_1^{EI}$',fontsize=14)
    ax1.set_ylabel(r'$\kappa_2^{EI}$',fontsize=14)
    # ax1.set_title(r'$J_E-J_I=$'+str(JEmI)+', $J_E+J_I=$'+str(JEpI)+'\n $J_E=$'+str(JE)+r',$J_I=$'+str(JI))
    # ax1.set_title(r'$a-b=$'+str(amb)+', $a+b=$'+str(apb)+'\n $a=$'+str(a)+r',$b=$'+str(b))
    ax1.set_title(r'$J_E-J_I=$'+str(JEmI)+', $a+b=$'+str(apb)+'\n $J_E=$'+str(JE)+r',$J_I=$'+str(JI)+',$a=$'+str(a)+',$b=$'+str(b))
    ax1.set_xlim([-3.0,3.0])
    ax1.set_ylim([-3.0,3.0])
    ax1.set_aspect('equal')
    plt.legend()
    plt.grid()

In [213]:
# Use kappa EI
# M and N
JEmIs,JEpIs=np.array([-0.5,0.5,1.0,1.5]),np.array([0.5,1.0,1.5,2.0])
for idxm, JEmI in enumerate(JEmIs):
    for idxp,JEpI in enumerate(JEpIs):
        amp,apb = -0.3,-0.3
        JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
        a,b=(apb+amb)/2.0,(apb-amb)/2.0
        # print('JE',JE)
        # a,b=-JE,JI
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
        M,N = np.array([[1,1],[1,-1]]),np.array([[JE,a],[-JI,b]])
        kappaei1,kappaei2=np.linspace(-3.0,3.0,301),np.linspace(-3.0,3.0,301)
        [kappaEI1,kappaEI2]=np.meshgrid(kappaei1,kappaei2)
        # Surface of rhs in ODEs
        XE,XI=kappaEI1*M[0,0]+kappaEI2*M[0,1],kappaEI1*M[1,0]+kappaEI2*M[1,1]
        rhs1,rhs2=N[0,0]*tanhpfunc(XE)+N[1,0]*tanhpfunc(XI),N[0,1]*tanhpfunc(XE)+N[1,1]*tanhpfunc(XI)
        # rhsE,rhsI=Jt[0,0]*np.tanh(XE)+Jt[0,1]*np.tanh(XI),Jt[1,0]*np.tanh(XE)+Jt[1,1]*np.tanh(XI)
        epsdis = 1*1e-4#((xE[1]-xE[0])/5.0)**2
        # finding the intersection
        distrhs1=(rhs1-kappaEI1)**2
        intersect1=np.where(distrhs1<epsdis)
        distrhs2=(rhs2-kappaEI2)**2
        intersect2=np.where(distrhs2<epsdis)

        ## plot intersection line
        fig=plt.figure()
        ax1=plt.axes()
        ax1.scatter(kappaei1[intersect1[1]],kappaei2[intersect1[0]],c='red',s=1,label=r'Nullcline $\kappa_1^{EI}$')
        # ax1.scatter(kappaei1[intersect2[1]],kappaei2[intersect2[0]],c='blue',s=1,label=r'Nullcline $\kappa_2^{EI}$')
        ax1.set_xlabel(r'$\kappa_1^{EI}$',fontsize=14)
        # ax1.set_ylabel(r'$\kappa_2^{EI}$',fontsize=14)
        ax1.set_title(r'$J_E-J_I=$'+str(JEmI)+', $J_E+J_I=$'+str(JEpI)+'\n $J_E=$'+str(JE)+r',$J_I=$'+str(JI))
        # ax1.set_title(r'$a-b=$'+str(amb)+', $a+b=$'+str(apb)+'\n $a=$'+str(a)+r',$b=$'+str(b))
        # ax1.set_title(r'$J_E-J_I=$'+str(JEmI)+', $a+b=$'+str(apb)+'\n $J_E=$'+str(JE)+r',$J_I=$'+str(JI)+',$a=$'+str(a)+',$b=$'+str(b))
        ax1.set_xlim([-3.0,3.0])
        ax1.set_ylim([-3.0,3.0])
        ax1.set_aspect('equal')
        plt.legend()
        plt.grid()


In [211]:
ambs,apbs=np.array([2.50]),np.array([-0.5,-0.05,0.05,0.5])
for idxm, amb in enumerate(ambs):
    for idxp,apb in enumerate(apbs):
        JE,JI=(JEpI+JEmI)/2.0,(JEpI-JEmI)/2.0
        a,b=(apb+amb)/2.0,(apb-amb)/2.0
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]+a,Jt[0,1]+b,Jt[1,0]-a,Jt[1,1]-b
        M,N = np.array([[1,1],[1,-1]]),np.array([[JE,a],[-JI,b]])
        kappaei1,kappaei2=np.linspace(-3.0,3.0,301),np.linspace(-3.0,3.0,301)
        [kappaEI1,kappaEI2]=np.meshgrid(kappaei1,kappaei2)
        # Surface of rhs in ODEs
        XE,XI=kappaEI1*M[0,0]+kappaEI2*M[0,1],kappaEI1*M[1,0]+kappaEI2*M[1,1]
        rhs1,rhs2=N[0,0]*tanhpfunc(XE)+N[1,0]*tanhpfunc(XI),N[0,1]*tanhpfunc(XE)+N[1,1]*tanhpfunc(XI)
        # rhsE,rhsI=Jt[0,0]*np.tanh(XE)+Jt[0,1]*np.tanh(XI),Jt[1,0]*np.tanh(XE)+Jt[1,1]*np.tanh(XI)
        epsdis = 1*1e-4#((xE[1]-xE[0])/5.0)**2
        # finding the intersection
        distrhs1=(rhs1-kappaEI1)**2
        intersect1=np.where(distrhs1<epsdis)
        distrhs2=(rhs2-kappaEI2)**2
        intersect2=np.where(distrhs2<epsdis)

        ## plot intersection line
        fig=plt.figure()
        ax1=plt.axes()
        ax1.scatter(kappaei1[intersect1[1]],kappaei2[intersect1[0]],c='red',s=1,label=r'Nullcline $\kappa_1^{EI}$')
        # ax1.scatter(kappaei1[intersect2[1]],kappaei2[intersect2[0]],c='blue',s=1,label=r'Nullcline $\kappa_2^{EI}$')
        ax1.set_xlim([-3.0,3.0])
        ax1.set_ylim([-3.0,3.0])
        ax1.set_xlabel(r'$\kappa_1^{EI}$',fontsize=14)
        # ax1.set_ylabel(r'$\kappa_2^{EI}$',fontsize=14)
        ax1.set_title(r'$J_E-J_I=$'+str(JEmI)+', $J_E+J_I=$'+str(JEpI)+'\n $J_E=$'+str(JE)+r',$J_I=$'+str(JI))
        # ax1.set_title(r'$a-b=$'+str(amb)+', $a+b=$'+str(apb)+'\n $a=$'+str(a)+r',$b=$'+str(b))
        # ax1.set_title(r'$J_E-J_I=$'+str(JEmI)+', $a+b=$'+str(apb)+'\n $J_E=$'+str(JE)+r',$J_I=$'+str(JI)+',$a=$'+str(a)+',$b=$'+str(b))

        ax1.set_aspect('equal')
        plt.legend()
        plt.grid()


In [10]:
plt.figure()
c=plt.contour(kappaEI1,kappaEI2,rhs1,20,cmap='rainbow')
plt.clabel(c,inline=True,fontsize=10)
d=plt.contour(kappaEI1,kappaEI2,kappaEI1,8,colors='BLACK')
plt.clabel(d,inline=True,fontsize=10)

plt.figure()
c=plt.contour(kappaEI1,kappaEI2,rhs2,20,cmap='rainbow')
plt.clabel(c,inline=True,fontsize=10)
d=plt.contour(kappaEI1,kappaEI2,kappaEI2,8,colors='BLACK')
plt.clabel(d,inline=True,fontsize=10)

<a list of 8 text.Text objects>

# Part 2 large Random Matrix
* large matrix with $N_E$ excitatory neurons and $N_I$ inhibitory neurons
* rank 1 and then rank 2 structure 
* different decomposition, SVD and EI, what are the same, what are different
* knowledge learnt from 2by2 case can be spread to the larger case